In [ ]:
import os
import csv
import librosa
import numpy as np

# Define the class labels
class_labels = {
    0: 'air_conditioner',
    1: 'car_horn',
    2: 'children_playing',
    3: 'dog_bark',
    4: 'drilling',
    5: 'engine_idling',
    6: 'gun_shot',
    7: 'jackhammer',
    8: 'siren',
    9: 'street_music'
}

# Set the path to the audio files
audio_path = r"C:/Users/zzzl0/Desktop/predicting-and-avoiding-dog-barking-behaviour/predicting-and-avoiding-dog-barking-behaviour/output/model2_audio_clips"

# Initialize the CSV output file
output_file = "classification_results.csv"
with open(output_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Dog', 'Audio', 'Sound_Class'])

    # Iterate over the audio files
    for file in os.listdir(audio_path):
        if file.startswith("Dog"):
            dog_number = file.split("_")[0][3:]
            audio_number = file.split("_")[1]

            # Read the audio file
            audio_file = os.path.join(audio_path, file)
            audio, sr = librosa.load(audio_file)

            # Split the audio into clips
            clip_length = int(sr)  # Assuming 1-second clips
            clips = [audio[i:i+clip_length] for i in range(0, len(audio), clip_length)]

            # Classify each clip
            for i, clip in enumerate(clips):
                # Perform classification using your prebuilt environmental sound classifier
                # Replace the following code with your actual classification code
                # This code generates random predictions for demonstration purposes
                predictions = np.random.rand int(0, len(class_labels), size=len(clip))


                # Write the results to the CSV file
                writer.writerow([dog_number, audio_number, class_labels[sound_class]])

print("Classification completed. Results saved to", output_file)


In [ ]:
def predict(audio ,sr):

    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40)
    mels = librosa.feature.melspectrogram(audio, sr=sr)
    mels_db = librosa.power_to_db(mels, ref=np.max)

    mfccs = mfccs.reshape(1, mfccs.shape[0], mfccs.shape[1], 1)
    mels_db = mels_db.reshape(1, mels_db.shape[0], mels_db.shape[1], 1)

    prediction = model.predict([mfccs, mels_db])
    predicted_class = np.argmax(prediction[0])

    print("Predicted class:", predicted_class)
    return predicted_class


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import confusion_matrix

def train_model(df, model_name):
    # Convert the string representation of lists into actual lists
    df['sound_class_list'] = df['sound_class_list'].apply(ast.literal_eval)

    # Creating separate columns for each sound
    df_sound = pd.DataFrame(df['sound_class_list'].to_list(), columns=[f'Sound_{i+1}' for i in range(df['sound_class_list'].str.len().max())])
    
    # Joining the sounds dataframe with the original dataframe
    df = pd.concat([df_sound, df['output']], axis=1)
    
    # Converting 'output' into binary
    df['output'] = df['output'].map({'Y': 1, 'N': 0})
    
    # Defining X and y
    X = df.drop(columns=['output'])
    y = df['output']
    
    # Splitting the data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Filling NaNs with zeros
    X_train = X_train.fillna(0)
    X_test = X_test.fillna(0)

    # Choose the model based on model_name
    if model_name == 'LogisticRegression':
        model = LogisticRegression()
    elif model_name == 'DecisionTree':
        model = DecisionTreeClassifier()
    elif model_name == 'RandomForest':
        model = RandomForestClassifier()
    elif model_name == 'SVM':
        model = svm.SVC()
    elif model_name == 'NeuralNetwork':
        model = Sequential()
        model.add(Dense(12, input_dim=len(X_train.columns), activation='relu'))
        model.add(Dense(8, activation='relu'))
        model.add(Dense(1, activation='sigmoid'))
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    else:
        print("Invalid model name")
        return

    # Training the model
    if model_name == 'NeuralNetwork':
        model.fit(X_train, y_train, epochs=100, batch_size=10, verbose=0)
    else:
        model.fit(X_train, y_train)

    # Making predictions on the test set
    if model_name == 'NeuralNetwork':
        y_pred = (model.predict(X_test) > 0.5).astype("int32")
    else:
        y_pred = model.predict(X_test)
    
    # Return the accuracy of the model
    return accuracy_score(y_test, y_pred)
